# IMPORTANT
This notebook is to demonstrate how EDA of this notebook can be done in SQL rather than just Pandas and for the author's (Jacob Zimmerman's) learning purposes. It is **NOT** the full notebook. Most code that was unnecessary to completing the SQL versions of EDA tasks has been ommitted. The full notebook, containing modeling, extensive analyses, and detailed visualizations, is located [here](https://github.com/jakezimm12/Open-Food-Facts-EDA-and-Nutritional-Value-Modeling). Note, I do not use SQL to redo every EDA operation or to redo any of the cleaning because it was oftentimes quite simple. To just see the SQL, skip the 'Cleaning the Data' section and scroll to the 'EDA' section (or click the link in the table of contents on the left in Collab).

# CIS 5450 Group Project
Note, the SQL versions of EDA tasks were written by Jacob Zimmerman using PandasSQL and SQLite.

By Alberto Gaytan, Jacob Zimmerman, and Jingxian Qiu <br>
15 December 2022

### Setup
Please copy "entire_brand_df.csv" to your general "My Drive" (not within a folder), and make sure it maintains its name: https://drive.google.com/file/d/1--CA88tj8XvEnb-BAZmbkkF_7xwu2Aya/view?usp=sharing



We are using a Kaggle dataset.

To get the data in here:
1. Go to this [Kaggle link](https://www.kaggle.com) and create a Kaggle account (unless you already have an account)
2. Go to Account and click on "Create New API Token" to get the API key in the form of a json file `kaggle.json`
3. Upload the `kaggle.json` file to the default location in your Google Drive (Don't upload the json file into any _specific_ folder.).

### Motivation: Predicting Nutrition Scores from Open Food Facts Data
With rates of health related issues stemming from a poor diet increasing in recent times, awareness of the need to place greater importance on nutrition has followed. The Open Food Facts database, being open to the public, is likely to be a primary candidate for those looking to analyze what foods they should maintain in their diet and which to cut out. The primary goal of this notebook is to uncover insights about the nutritional value of a product based on its characteristics and to figure out the best way to predict a new product's nutrition score given that we have certain (Open-Food-Facts-like) features about the new product.

Note, we use the nutritional scores provided in the dataset as our measure of nutritional value. A higher nutrition score means less healthy (officially, it means the product has "less nutritional quality," but we use "healthy" and "higher nutritional quality" interchangeably throughout this analysis, which we believe is a reasonable assumption to make). The score ranges from -15 to 40. Learn more here: https://get.apicbase.com/nutri-score-science-based-nutritional-value-labelling-system/


In [312]:
!pip install pandasql
!pip3 install py_stringsimjoin

import json
import glob
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import matplotlib.pyplot as plt
from lxml import etree
import urllib
import urllib.request
import seaborn as sns
import py_stringsimjoin as ssj
import py_stringmatching as sm
from matplotlib import cm
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, Ridge, LassoCV, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.express as px
import pandasql as ps #SQL on Pandas Dataframe
import altair as alt
import pandasql as ps #SQL on Pandas Dataframe

alt.data_transformers.disable_max_rows()

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


DataTransformerRegistry.enable('default')

In [313]:
!apt update

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state infor

In [314]:
# Run this cell to mount your drive (you will be prompted to sign in)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [315]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [316]:
# Create the kaggle directory and read the uploaded kaggle.json file
# (NOTE: Do NOT run this cell more than once unless restarting kernel)
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [317]:
# Read the uploaded kaggle.json file
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [318]:
# Download dataset
!kaggle datasets download -d openfoodfacts/world-food-facts

world-food-facts.zip: Skipping, found more recently modified local copy (use --force to force download)


In [319]:
# Unzip folder in Colab content folder
!unzip /content/world-food-facts

Archive:  /content/world-food-facts.zip
replace en.openfoodfacts.org.products.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: en.openfoodfacts.org.products.tsv  


In [320]:
# Read the csv file and save it to a dataframe called `df_airbnb`
ff_df = pd.read_csv("en.openfoodfacts.org.products.tsv", sep='\t', low_memory=False)
ff_df.head(3)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000000003087/farine-de-ble-noir-ferme-t-y-r-nao,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-completed, en:ingredients-to-be-completed, en:expir...","en:to-be-completed,en:nutrition-facts-to-be-completed,en:ingredients-to-be-completed,en:expirati...","To be completed,Nutrition facts to be completed,Ingredients to be completed,Expiration date to b...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000000004530/banana-chips-sweetened-whole,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489

In [321]:
# CHECKING DIFFERENCE BETWEEN UK AND FRANCE NUTRITION SCORES
is_same = ff_df.dropna(subset=['nutrition-score-uk_100g', 'nutrition-score-fr_100g'])
is_same = is_same['nutrition-score-uk_100g'] == is_same['nutrition-score-fr_100g']
# print(len(is_same))

count=0
for entry in is_same:
  if entry == True:
    count += 1
# print(count)
print("UK and France versions of nutrition scores are the same: " + str(100 * count / len(is_same)) + "% of the time")

UK and France versions of nutrition scores are the same: 94.87436042314091% of the time


# Cleaning the Data

In [322]:
# Check the basic information of the dataframe
ff_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356027 entries, 0 to 356026
Columns: 163 entries, code to water-hardness_100g
dtypes: float64(107), object(56)
memory usage: 442.8+ MB


## Dropping Columns with a Significant Amount of Nulls
Most of the 163 columns in this dataset have a significant amount of nulls. Here I drop columns with more than 30% nulls.

In [323]:
# Drop columns with more than 30% nulls
perc = 30.0 # max percent of null values we are allowing
min_count =  int(((100-perc)/100)*ff_df.shape[0] + 1)
ff_df = ff_df.dropna( axis=1, thresh=min_count)

## Dropping Unnecessary Columns

Next, I drop many columns because they will not add information to our analysis or are a close duplicate of another column. I provide reasoning for dropped columns below where it is not obvious:
*   'creator' is the organization that created the entry
*   'created_t', 'created_datetime', 'last_modified_t', and 'last_modified_datetime' all have to do with when the row entry was created.
*   The 'brands' column is the same as the 'brands_tags' column just with different formatting. Removed 'brands_tags'.
*   'countries' and 'countries_tags' is the same as 'countries_en' with different formatting.
*   'additives' is similar to 'ingredients_text' with different formatting. Note there is a number of additives column that we didn't drop.
*   'ingredients_from_palm_oil_n' and 'ingredients_that_may_be_from_palm_oil_n' is almost always 0 and, upon inspection, is usually not accurate according to the ingredients column (many times the ingredients has palm oil but is not reflected in the 'ingredients_from_palm_oil_n' or the 'ingredients_that_may_be_from_palm_oil_n' columns.
*   The 'states' columns have to do with what information still needs to be added to a respective entry.

In [324]:
# Drop unnecssary columns
ff_df = ff_df.reset_index().drop(columns=['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 
                            'last_modified_datetime', 'brands_tags', 'countries', 'countries_tags', 'additives', 
                            'ingredients_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_n', 'states',
                            'states_tags', 'states_en'])

Let's see what columns are left.

In [325]:
# Check again the updated dataframe
print("Open Food Facts Dataframe columns this far: ")
print(ff_df.columns)

Open Food Facts Dataframe columns this far: 
Index(['index', 'product_name', 'brands', 'countries_en', 'ingredients_text',
       'additives_n', 'nutrition_grade_fr', 'energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g', 'nutrition-score-fr_100g',
       'nutrition-score-uk_100g'],
      dtype='object')


Rename some columns for easier reading.

In [326]:
# Dataframe renaming
ff_df = ff_df.rename(columns={'countries_en':'countries', 'nutrition-score-fr_100g':'nutrition_score', 'nutrition-score-uk_100g':'nutrition_score_uk',
                              'brand-tags':'brand','nutrition_grade_fr':'nutrition_grade', 'ingredients_text':'ingredients',
                              'energy_100g':'energy', 'fat_100g':'fat', 'saturated-fat_100g':'saturated_fat', 'carbohydrates_100g':'carbohydrates', 
                              'sugars_100g':'sugars','proteins_100g':'proteins', 'salt_100g':'salt', 'sodium_100g':'sodium'})

## Cleaning 'countries' and 'brands' Columns
We will be exploring the 'countries' and 'brands' columns later. The 'countries' column specificies the country in which the product is available in English, and the 'brands' column denotes the comapny that made the product. However, these columns sometimes have multiple values represented as comma-separated strings. Let's change these columns to hold lists instead of strings (including the rows with only one value).

However, we need to do some extra cleaning for 'brands'. We have to take out leading spaces because this column is comma-space-separeated. Also, some of the names are just "nan" denoting null. So, we have to make these null.

In [327]:
# Process rows with 'nan' brands
def take_out_string_nans(brands_list):
  for i, brand in enumerate(brands_list):
    if brand == 'nan':
      brands_list[i] = None
  return brands_list

# Str2List for column countries and brands
ff_df['countries'] = ff_df['countries'].apply(lambda countries_str: str(countries_str).split(','))
ff_df['brands'] = ff_df['brands'].apply(lambda brands_str: [brand.strip().strip("\"") for brand in str(brands_str).split(',')])
ff_df['brands'] = ff_df['brands'].apply(lambda brands_list: take_out_string_nans(brands_list))

Quick check:

In [328]:
## Check again the updated dataframe

print("Couple entries with multiple values in the 'countries' column to validate our work: ")
count = 0
for countries_list in ff_df['countries']:
  if len(countries_list) > 1:
    count += 1
    print(countries_list)
    if count == 5:
      break

print()

print("Couple entries with multiple values in the 'brands' column to validate our work: ")
count = 0
for brands_list in ff_df['brands']:
  if len(brands_list) > 1:
    count += 1
    print(brands_list)
    if count == 5:
      break

Couple entries with multiple values in the 'countries' column to validate our work: 
['France', 'United States']
['France', 'United States']
['France', 'United Kingdom']
['France', 'United Kingdom']
['France', 'United Kingdom']

Couple entries with multiple values in the 'brands' column to validate our work: 
['Wholesome', 'Wholesome Sweeteners']
['Wholesome', 'Wholesome Sweeteners']
['Kettle', 'Kettle Foods']
['Kettle', 'Kettle Foods']
['Kettle', 'Kettle Foods']


## General Cleaning

Change product names to lowercase for later use.

In [329]:
# Change product name to lower case
ff_df['product_name'] = ff_df['product_name'].str.lower() # first lower case 

# EDA

## Exploring France and UK Nutrition Scores

### First, in Pandas.
Getting French and UK nutrition scores where they are not equal (each row will contain a product's corresponding French and UK nutritional scores). We chose to use the French score after this analysis (rationale explained in the original notebook).

In [374]:
uk_scores = ff_df[['nutrition_score_uk','nutrition_score']].dropna()['nutrition_score_uk'].to_frame().rename(columns={'nutrition_score_uk':'nutrition_score'})
uk_scores['version'] = 'uk'
fr_scores = ff_df[['nutrition_score','nutrition_score_uk']].dropna()['nutrition_score'].to_frame()
fr_scores['version'] = 'fr'
scores = pd.concat([uk_scores,fr_scores]).reset_index(drop=True)

### Now, in SQL.

In [375]:
sql_ff_df = ff_df.drop(columns=['brands', 'countries']) # for now take these two cols out because SQLite doesn't support their types, we'll deal with this later
sql_scores_query = """
WITH uk_scores AS (
  SELECT nutrition_score AS nutrition_score, 'uk' AS version
  FROM sql_ff_df
  WHERE nutrition_score_uk IS NOT NULL
),
fr_scores AS (
  SELECT nutrition_score, 'fr' AS version
  FROM sql_ff_df
  WHERE nutrition_score IS NOT NULL
)
SELECT nutrition_score, version
FROM uk_scores
UNION ALL
SELECT nutrition_score, version
FROM fr_scores;
"""
sql_scores = ps.sqldf(sql_scores_query, locals())

### Compare Pandas and SQL Results



In [376]:
print("First Three Entries:\n")
print("Pandas")
print(scores.head(3))
print()
print("SQL")
print(sql_scores.head(3))
print()
print("Lengths:\n")
print("Pandas: " + str(len(scores)))
print("SQL: " + str(len(sql_scores)))


First Three Entries:

Pandas
   nutrition_score version
0             14.0      uk
1              0.0      uk
2             12.0      uk

SQL
   nutrition_score version
0             14.0      uk
1              0.0      uk
2             12.0      uk

Lengths:

Pandas: 509712
SQL: 509712




Now, we focus on the entries with different scores in France and UK nutrition scoring systems.

### First, in Pandas.
Getting the difference and absolute difference between the French and UK nutrition scores for each product where the scores don't match, as well as an indication column denoting whether the UK nutrition score was higher than the French.

In [377]:
# Compare nutrition scores in different system
compare_scores = ff_df[ff_df['nutrition_score']!=ff_df['nutrition_score_uk']][['product_name','brands','nutrition_score','nutrition_score_uk']].dropna()
compare_scores['diff'] = compare_scores['nutrition_score'] - compare_scores['nutrition_score_uk']
compare_scores['higher_uk'] = compare_scores['diff'].apply(lambda x: True if x<0 else False)
compare_scores['abs_diff'] = compare_scores['diff'].apply(lambda x: abs(x))
compare_scores.describe()

,nutrition_score,nutrition_score_uk,diff,abs_diff
count,12999.000000,12999.000000,12999.000000,12999.000000
mean,11.107624,7.499038,3.608585,7.499808
std,5.812308,9.448135,7.528315,3.667011
min,-10.000000,-9.000000,-14.000000,1.000000
25%,7.000000,0.000000,-5.000000,5.000000
50%,12.000000,2.000000,6.000000,7.000000
75%,15.000000,18.000000,10.000000,10.000000
max,40.000000,31.000000,21.000000,21.000000


### Now, in SQL.

In [378]:
sql_ff_df = ff_df.drop(columns=['brands', 'countries']) # (recreating so readers know what this dataframe is)
                                                        # for now take these two cols out because SQLite doesn't
                                                        # support their types, we'll deal with this later

sql_compare_scores_query = """
SELECT product_name,
        nutrition_score,
        nutrition_score_uk,
        nutrition_score - nutrition_score_uk AS diff,
        CASE
          WHEN nutrition_score_uk - nutrition_score > 0 THEN 'True'
          ELSE 'False'
        END AS higher_uk,
        ABS(nutrition_score_uk - nutrition_score) AS abs_diff
FROM  sql_ff_df
WHERE nutrition_score != nutrition_score_uk
  AND nutrition_score IS NOT NULL
  AND nutrition_score_uk IS NOT NULL
  AND product_name IS NOT NULL
;
"""

sql_compare_scores = ps.sqldf(sql_compare_scores_query, locals())

### Comparison of Pandas and SQL Results

In [379]:
pd.set_option('expand_frame_repr', False)

In [380]:
print("First Three Entries:\n")
print("Pandas")
print(compare_scores.reset_index().drop(columns=['index']).head(3)) # changing around indices a bit here for symmetry
print()
print("SQL (Note, brands is taken out because SQLite doesn't support its type)")
print(sql_compare_scores.head(3))
print()
print("Stats:\n")
print("Pandas")
print(compare_scores.describe())
print("SQL Stats")
print(compare_scores.describe())

First Three Entries:

Pandas
                            product_name          brands  nutrition_score  nutrition_score_uk  diff  higher_uk  abs_diff
0                              root beer           [A&W]             18.0                 3.0  15.0      False      15.0
1  thé noir aromatisé violette et fleurs  [Alice Délice]              2.0                 0.0   2.0      False       2.0
2                  pepsi, nouveau goût !         [Pepsi]             13.0                 2.0  11.0      False      11.0

SQL (Note, brands is taken out because SQLite doesn't support its type)
                            product_name  nutrition_score  nutrition_score_uk  diff higher_uk  abs_diff
0                              root beer             18.0                 3.0  15.0     False      15.0
1  thé noir aromatisé violette et fleurs              2.0                 0.0   2.0     False       2.0
2                  pepsi, nouveau goût !             13.0                 2.0  11.0     False      11.

## Distribution of Product Availability Across Country
Let's visualize the distribution of the products in this dataset across countries (recall the "countries" column represents the countries that the product is available to be purchased in).

Let's project two columns and explode the lists we made in the 'countries' column to make this easier.

In [381]:
# Explode list of countries to rows
exploded_ff_df = ff_df[['countries', 'nutrition_score']].explode('countries')

Let's see what countries we are left with.

In [382]:
# replace easy string inconsistencies

exploded_ff_df['countries'].unique()

array(['France', 'United States', 'United Kingdom', 'Canada',
       'Switzerland', 'Spain', 'Germany', 'Slovenia', 'Bangladesh',
       'Italy', 'Romania', 'Australia', 'Portugal', 'Belgium',
       'Netherlands', 'Guadeloupe', 'Dominican Republic', 'Ireland',
       'Réunion', 'Taiwan', 'Brazil', 'Mexico', 'Saudi Arabia',
       'United Arab Emirates', 'Iraq', 'Kuwait', 'Moldova',
       'New Caledonia', 'nan', 'Singapore', 'Finland', 'Sweden',
       'French Polynesia', 'New Zealand', 'Thailand', 'Panama',
       'Costa Rica', 'Scotland', 'Trinidad and Tobago', 'China',
       'Hong Kong', 'Luxembourg', 'Lebanon', 'Denmark', 'Belgie',
       'Nederland', 'Malaysia', 'Guyana', 'Serbia', 'Martinique',
       'Barbados', 'Aruba', 'Norway', 'Haiti', 'El Salvador', 'Bahrain',
       'Guatemala', 'Morocco', 'Oman', 'South Korea', 'Chile', 'Cuba',
       'Japan', 'Israel', 'Philippines', 'United-states-of-america',
       'Austria', 'Guinea', 'Jordan', 'Russia',
       'Saint Pierre and Mi

Here, we can see there are non-english (some are in French, denoted by 'fr:(country name)' and some use non-alphabetic characters) and non-standardized (Ex. 'United-states-of-america' instead of 'United States of America') entries. The vast majority of entries have capitalized, english names of countries with spaces between parts of the name as necessary. So, we just remove non-english entries and reformat non-space-separated (dash separated) entries. 

### First, in Pandas.

In [383]:
# Country cleaning
exploded_ff_df['countries'] = exploded_ff_df['countries'].apply(lambda country: country.replace('-', ' '))
cleaned_exploded_ff_df = exploded_ff_df[~exploded_ff_df.countries.str.contains(r'[^\w\s]')] # remove names with characters that are non-letters or whitespace
cleaned_exploded_ff_df = cleaned_exploded_ff_df[cleaned_exploded_ff_df.countries.map(lambda x: x.isascii())] # remove names with non-english characters
# sorted(cleaned_exploded_ff_df['countries'].unique())

Let's check to see how many rows we removed.

In [384]:
# Count the dropped rows
print("Lost " + str(exploded_ff_df.shape[0] - cleaned_exploded_ff_df.shape[0]) + " of " + str(exploded_ff_df.shape[0]) + " rows")

Lost 472 of 363872 rows


Additionally, let's remove 'Virgin Islands of the United States' (not compatible with our visualization), 'nan', 'Ap' (an abbrevation for Asia/Pacific Region), and 'World' (We didn't do this above so we could see how many rows were lost before this step). Also, there are no variations of country names except "United States" and "United states of america." We handle that here.


In [385]:
# Country cleaning
cleaned_exploded_ff_df = cleaned_exploded_ff_df[(cleaned_exploded_ff_df['countries'] != 'nan') & (cleaned_exploded_ff_df['countries'] != 'World') & (cleaned_exploded_ff_df['countries'] != 'Ap') & (cleaned_exploded_ff_df['countries'] != 'Virgin Islands of the United States')]
cleaned_exploded_ff_df.loc[cleaned_exploded_ff_df["countries"] == "United states of america"] = 'United States'
None

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Now, in SQL. Note, I didn't explode in SQL as 'countries' is not in a supported SQLite data type.

In [398]:
sql_cleaned_exploded_ff_df_query = """
SELECT REPLACE(countries, '-', ' ') AS countries,
  nutrition_score
FROM exploded_ff_df
;
"""
sql_cleaned_exploded_ff_df = ps.sqldf(sql_cleaned_exploded_ff_df_query, locals())

sql_cleaned_exploded_ff_df_query2 = """
SELECT *
FROM sql_cleaned_exploded_ff_df
WHERE NOT REGEXP(countries, 'r[^\w\s]')
  AND countries NOT LIKE '%nan%'
  AND countries NOT LIKE '%World%'
  AND countries NOT LIKE '%Ap%'
  AND countries NOT LIKE '%Virgin Islands of the United States%'
;
"""
sql_cleaned_exploded_ff_df = ps.sqldf(sql_cleaned_exploded_ff_df_query2, locals())

### Comparing Pandas and SQL Results

In [400]:
print("First Three Entries:\n")
print("Pandas")
print(cleaned_exploded_ff_df.head(3))
print()
print("SQL")
print(sql_cleaned_exploded_ff_df.head(3))
print()
print("Lengths:\n")
print("Pandas: " + str(len(cleaned_exploded_ff_df)))
print("SQL: " + str(len(sql_cleaned_exploded_ff_df)))
print()
print("The difference is because I did not take out the non-ASCII countries in SQL.")

First Three Entries:

Pandas
       countries nutrition_score
0         France             NaN
1  United States            14.0
2  United States             0.0

SQL
       countries  nutrition_score
0         France              NaN
1  United States             14.0
2  United States              0.0

Lengths:

Pandas: 363115
SQL: 363171

The difference is because I did not take out the non-ASCII countries in SQL.


## Does being "organic" mean that a product is Healthier? How about "diet" or "lite?"
One common theory is that companies often use keywords like "organic" in a product's name to lead customers into believing a product is healthy. Here, we aim to see if this idea has some data-driven truth behind it.

### First, in Pandas.

Here we are getting the nutrition scores of products containing the aforementioned keywords. We separate them into separate tables for ease of analysis (analysis in the original notebook).

In [388]:
# Filter the products with different keywords
org_df = ff_df[(ff_df['product_name'] != None) & (ff_df.product_name.str.contains('organic'))][['product_name', 'nutrition_score']] # also taking the product name here in case reader would like to verify or for later analysis
diet_df = ff_df[(ff_df['product_name'] != None) & (ff_df.product_name.str.contains('diet'))][['product_name', 'nutrition_score']]
lite_df = ff_df[(ff_df['product_name'] != None) & ((ff_df.product_name.str.contains('lite')) | (ff_df.product_name.str.contains('light')))][['product_name', 'nutrition_score']]

In [389]:
healthy_keyword_nut_scores = pd.concat([org_df.rename(columns={'nutrition_score':'Organic Nutrition Scores'})['Organic Nutrition Scores'],
                                        diet_df.rename(columns={'nutrition_score':'Diet Nutrition Scores'})['Diet Nutrition Scores'],
                                        lite_df.rename(columns={'nutrition_score':'Lite/Light Nutrition Scores'})['Lite/Light Nutrition Scores']],
                                        axis=1)

### Now, in SQL. I realize I could have condensed this without using UNIONs, but I wanted to follow the logic of creating separate tables as in the Pandas version above.

In [390]:
sql_ff_df = ff_df.drop(columns=['brands', 'countries']) # (recreating so readers know what this dataframe is)
                                                        # for now take these two cols out because SQLite doesn't
                                                        # support their types, we'll deal with this later

sql_healthy_keyword_nut_scores_query = """
SELECT product_name,
  nutrition_score
FROM sql_ff_df
WHERE product_name IS NOT NULL
AND product_name LIKE '%organic%'

UNION ALL

SELECT product_name,
  nutrition_score
FROM sql_ff_df
WHERE product_name IS NOT NULL
AND product_name LIKE '%diet%'

UNION ALL

SELECT product_name,
  nutrition_score
FROM sql_ff_df
WHERE product_name IS NOT NULL
AND (product_name LIKE '%lite%' OR product_name LIKE '%light%')

;
"""
sql_healthy_keyword_nut_scores = ps.sqldf(sql_healthy_keyword_nut_scores_query, locals())

### Compare the Pandas and SQL Results



In [401]:
print("First Three Entries:\n")
print("Pandas")
print(healthy_keyword_nut_scores.head(3))
print()
print("SQL")
print(sql_healthy_keyword_nut_scores.head(3))
print()
print("Lengths:\n")
print("Pandas: " + str(len(healthy_keyword_nut_scores)))
print("SQL: " + str(len(sql_healthy_keyword_nut_scores)))
print()
print("Note, I just combined all the Nutrition Scores into one column in SQL (most entries are not in multiple columns). I typically would use a FULL JOIN to get all columns, but full joins are not supported in SQLite.")

First Three Entries:

Pandas
   Organic Nutrition Scores  Diet Nutrition Scores  Lite/Light Nutrition Scores
3                      12.0                    NaN                          NaN
4                       NaN                    NaN                          NaN
6                       NaN                    NaN                          NaN

SQL
                    product_name  nutrition_score
0         organic salted nut mix             12.0
1                organic polenta              NaN
2  organic long grain white rice              NaN

Lengths:

Pandas: 14106
SQL: 14215

Note, I just combined all the Nutrition Scores into one column in SQL (most entries are not in multiple columns). I typically would use a FULL JOIN to get all columns, but full joins are not supported in SQLite.


## Exploring Nutrients

Except for nutrition scores, our dataset provides information about nutrients, including energy, fat, saturated fat, carbohydrates, sugars, proteins, salt, and sodium. Some nutrients are subtypes of main nutrients. In our dataset, we define main nutrients as fat, carbohydrates, proteins, and salt. Note that energy is reported in "kJ per 100g", and other nutrients are reported in "g per 100g".

In [392]:
# Check Nutrients
nutrition_df = ff_df[['nutrition_score','energy', 'fat', 'saturated_fat', 'carbohydrates', 'sugars', 'proteins', 'salt', 'sodium']].dropna()
nutrition_df['main_nutrients'] = nutrition_df['fat'] + nutrition_df['carbohydrates'] + nutrition_df['proteins'] + nutrition_df['salt']

,nutrition_score,energy,fat,saturated_fat,carbohydrates,sugars,proteins,salt,sodium,main_nutrients
count,238533.000000,238533.000000,238533.000000,238533.000000,238533.000000,238533.000000,238533.000000,238533.000000,238533.000000,238533.000000
mean,9.176877,1172.075351,13.367942,4.913337,32.433032,14.794184,7.771801,1.613533,0.635250,55.186309
std,9.000967,768.383136,16.322374,7.441607,28.257215,19.693996,8.177840,132.369008,52.113783,136.336025
min,-15.000000,0.000000,0.000000,0.000000,0.000000,-17.860000,-3.570000,0.000000,0.000000,0.000000
25%,1.000000,448.000000,0.880000,0.000000,6.670000,1.200000,1.790000,0.100000,0.039370,23.550820
50%,10.000000,1176.000000,7.140000,1.790000,22.470000,5.000000,5.650000,0.635000,0.250000,54.087220
75%,16.000000,1711.000000,21.430000,7.100000,58.000000,22.580000,10.710000,1.353820,0.533000,88.101600
max,40.000000,22000.000000,200.000000,100.000000,165.000000,145.000000,305.000000,64312.800000,25320.000000,64384.950000


We need to do some additional data cleaning based on the following data validation methods:

* Any entries with more than 100g of total main nutrients are invalid data.
* The amount of nutrients should not be less than 0.
* The amount of any subtypes of the main nutrients is less than the amount of the main nutrient. For example, an entry is invalid when the amount of saturated fat is more than the amount of fat.

### First, with Pandas.

In [393]:
# Data cleaning: Drop invalid rows
invalid = nutrition_df[(nutrition_df['main_nutrients']>100) | (nutrition_df['saturated_fat']>nutrition_df['fat']) |
                       (nutrition_df['sugars']>nutrition_df['carbohydrates']) | (nutrition_df['sodium']>nutrition_df['salt'] ) |
                       (nutrition_df['sugars']<0) | (nutrition_df['proteins']<0)]

### Now, with SQL.

In [394]:
sql_nutrition_df = ff_df[['nutrition_score','energy', 'fat', 'saturated_fat', 'carbohydrates', 'sugars', 'proteins', 'salt', 'sodium']].dropna()
sql_nutrition_df['main_nutrients'] = sql_nutrition_df['fat'] + sql_nutrition_df['carbohydrates'] + sql_nutrition_df['proteins'] + sql_nutrition_df['salt']

sql_nutrition_df_query = """
DELETE FROM sql_nutrition_df
WHERE main_nutrients > 100
  OR saturated_fat > fat
  OR sugars > carbohydrates
  OR sodium > salt
  OR sugars < 0
  OR proteins < 0
;
"""
ps.sqldf(sql_nutrition_df_query, locals())

### Compare Pandas and SQL results.

In [395]:
print("First Three Entries:\n")
print("Pandas")
print(nutrition_df.head(3))
print()
print("SQL")
print(sql_nutrition_df.head(3))
print()
print("Lengths:\n")
print("Pandas: " + str(len(nutrition_df)))
print("SQL: " + str(len(sql_nutrition_df)))

First Three Entries:

Pandas
   nutrition_score  energy    fat  saturated_fat  carbohydrates  sugars  proteins     salt  sodium  main_nutrients
1             14.0  2243.0  28.57          28.57          64.29   14.29      3.57  0.00000   0.000        96.43000
2              0.0  1941.0  17.86           0.00          60.71   17.86     17.86  0.63500   0.250        97.06500
3             12.0  2540.0  57.14           5.36          17.86    3.57     17.86  1.22428   0.482        94.08428

SQL
   nutrition_score  energy    fat  saturated_fat  carbohydrates  sugars  proteins     salt  sodium  main_nutrients
1             14.0  2243.0  28.57          28.57          64.29   14.29      3.57  0.00000   0.000        96.43000
2              0.0  1941.0  17.86           0.00          60.71   17.86     17.86  0.63500   0.250        97.06500
3             12.0  2540.0  57.14           5.36          17.86    3.57     17.86  1.22428   0.482        94.08428

Lengths:

Pandas: 238533
SQL: 238533


## The rest of this notebook, including more EDA, Models, Obstacles Encountered, Conclusion, and Potential Next Steps, are located in the official GitHub repository of this project: https://github.com/jakezimm12/Open-Food-Facts-EDA-and-Nutritional-Value-Modeling. This notebook is to demonstrate how some the EDA could be done in SQL rather than just Pandas as in the original notebook. It is also for Jacob Zimmerman's learning. :)